In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)



## Some background reading

###### Space Weather:
- [Introduction](https://ccmc.gsfc.nasa.gov/RoR_WWW/SWREDI/2016/SpaceWeatherIntro_Bootcamp_2016.pdf)
- [Understanding space weather](https://www.sciencedirect.com/science/article/pii/S0273117715002252)

###### Particle Precipitation:
Here are a few particle precipitation resources that I believe are most valuable to start with:
- Technical details of the observations: [Redmon et al., [2017]](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016JA023339)
- Creating particle precipitation models from these data: [Hardy et al., [1987]](https://doi.org/10.1029/JA090iA05p04229) and [Newell et al., [2009]](https://doi.org/10.1029/2009JA014326)
- Considered the 'state of the art' model: [OVATION PRIME](https://ccmc.gsfc.nasa.gov/models/modelinfo.php?model=Ovation%20Prime)



## Imports and utility functions


In [2]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import datetime
from os.path import isfile, join
from sys import getsizeof
import glob

from random import *

from sklearn import *

## Prepare data for ML exploration (read in full DB created from standard_ML_DB_preparation.ipynb)


In [3]:
file_load_df_cumulative = 'ML_DB_subsamp_ext_full_dfCumulative_complexHemisphereCombine.csv'
DMSP_DATA_DIR=''
df_cumulative = pd.read_csv(os.path.join(DMSP_DATA_DIR,file_load_df_cumulative))
df_cumulative = df_cumulative.sort_values(by=['ID_SC', 'Datetimes'])
df_cumulative = df_cumulative.set_index('Datetimes')
df_cumulative.index = pd.to_datetime(df_cumulative.index)

cols_to_drop_validation = [c for c in df_cumulative.columns if ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]
# cols_to_drop_validation = [c for c in df.columns if ('1min' in c) | ('3min' in c) | ('4min' in c) | ('5min' in c) | ('15min' in c) | ('newell' in c) | ('STD' in c) | ('AVG' in c) | ('SC_AACGM_LTIME'==c)]

df_cumulative = df_cumulative.drop(columns=cols_to_drop_validation)


In [4]:
df_cumulative.shape



(1947016, 149)

In [5]:
# Separate training and testing data
mask_val = [(df_cumulative.index.year == 2010) & (df_cumulative['ID_SC'].values==16)]
df_val = df_cumulative[mask_val[0]].copy(deep=True)
df_train = df_cumulative.copy(deep=True).drop( df_cumulative.index[mask_val[0]])
print('validation data shape = {}'.format(df_val.shape))
print('train data shape = {}'.format(df_train.shape))
print('NOTE: we will use CV on the train data below to define model training and testing data,\n  so have called the withheld data *validation* data here')

# Construct X and y
feature_cols = [c for c in df_cumulative.columns if not 'ELE' in c]
#print( (feature_cols))
#print(df_cumulative.columns)

X_val = df_val[feature_cols].copy(deep=True)
y_val = df_val['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
X_train = df_train[feature_cols].copy(deep=True)
y_train = df_train['ELE_TOTAL_ENERGY_FLUX'].copy(deep=True)
scaler_X = preprocessing.RobustScaler()
scaler_X = scaler_X.fit(X_train.values)
X_val_scaled = scaler_X.transform(X_val.values)
X_train_scaled = scaler_X.transform(X_train.values)

numFeatures = len(X_train.columns.to_list())
feature_labels = X_train.columns.to_list()
#print(numFeatures)

validation data shape = (55210, 149)
train data shape = (1838283, 149)
NOTE: we will use CV on the train data below to define model training and testing data,
  so have called the withheld data *validation* data here


In [6]:
y_train_erg = y_train.copy(deep=True) * (1.60218e-12)
y_val_erg = y_val.copy(deep=True) * (1.60218e-12)

y_train[y_train == 0] = 0.0001
y_val[y_val == 0] = 0.0001
y_train_log = np.log10(y_train.copy(deep=True))
y_val_log = np.log10(y_val.copy(deep=True))

%matplotlib inline  
import matplotlib.pyplot as plt
X = np.array(X_train_scaled, dtype=np.float32)
X_test = np.array(X_val_scaled, dtype=np.float32)

Y = np.array(y_train_log, dtype=np.float32)
X.shape
X[:,2].size
Y.size

1838283

In [7]:

# plt.figure(figsize=(20,20))
# #plt.scatter(X[:,3],Y)
# plt.plot(X[:1000,2])
# plt.plot(Y[:1000])

# #plt.show()



In [8]:
X_train_scaled.shape[1]
hist_len = 64

In [9]:
X_train_scaled_hist = np.zeros((X.shape[0], hist_len,22, 1), dtype=np.float32)
X_test_scaled_hist = np.zeros((X_test.shape[0], hist_len,22, 1), dtype=np.float32)

for i in range(hist_len,X.shape[0]):
    for j in range(22):    
        X_train_scaled_hist[i-hist_len,:,j,0]= X[i-hist_len:i,j]
        

for i in range(hist_len,X_test.shape[0]):
    for j in range(22):    
        X_test_scaled_hist[i-hist_len,:,j,0]= X_test[i-hist_len:i,j]

In [10]:
# results = modelhist.evaluate(X_train_scaled_hist)#, y_val_log, batch_size=128)
# plt.figure(figsize=(50,15))

# plt.plot(y_train_log.values[:1000])
# plt.plot(results[:1000])
# plt.show()

# results = modelhist.evaluate(X_test_scaled_hist)#, y_val_log, batch_size=128)
# plt.figure(figsize=(50,15))

# plt.plot(y_val_log.values[:1000])
# plt.plot(results[:1000])
# plt.show()

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, Input, LSTM, TimeDistributed, MaxPooling1D, Dropout
from keras.layers.merge import concatenate

import keras.backend as K

Using TensorFlow backend.


In [15]:
model = Sequential()
model.add(Conv1D(44, kernel_size=int(9), activation='relu', input_shape=(hist_len,22)))
model.add(MaxPooling1D())
model.add(Conv1D(44, kernel_size=int(5), activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(22, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))
#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_scaled_hist[:,:,:,0], y_train_log, validation_data=(X_test_scaled_hist[:,:,:,0], y_val_log),
          batch_size=8192, epochs=40)

Train on 1838283 samples, validate on 55210 samples
Epoch 1/40
1838283/1838283 [==============================] - 76s 41us/step - loss: 6.3755 - accuracy: 0.0000e+00 - val_loss: 2.4300 - val_accuracy: 0.0000e+00
Epoch 2/40
1838283/1838283 [==============================] - 76s 42us/step - loss: 1.3637 - accuracy: 0.0000e+00 - val_loss: 2.0420 - val_accuracy: 0.0000e+00
Epoch 3/40
1838283/1838283 [==============================] - 76s 41us/step - loss: 1.1931 - accuracy: 5.4399e-07 - val_loss: 1.8541 - val_accuracy: 0.0000e+00
Epoch 4/40
1838283/1838283 [==============================] - 75s 41us/step - loss: 1.0790 - accuracy: 5.4399e-07 - val_loss: 1.7309 - val_accuracy: 0.0000e+00
Epoch 5/40
1838283/1838283 [==============================] - 75s 41us/step - loss: 0.9832 - accuracy: 5.4399e-07 - val_loss: 1.5658 - val_accuracy: 0.0000e+00
Epoch 6/40
1838283/1838283 [==============================] - 76s 41us/step - loss: 0.9131 - accuracy: 5.4399e-07 - val_loss: 1.4561 - val_accuracy:

In [ ]:
print(X.shape,X_train_scaled_hist.shape)
X1=X.reshape((X.shape[0],148,1))
print(X1.shape,X_train_scaled_hist.shape)
X_test1=X_test.reshape((X_test.shape[0],148,1))

X_train_scaled_hist1=X_train_scaled_hist.reshape((X_train_scaled_hist.shape[0],hist_len,22))
X_test_scaled_hist1=X_test_scaled_hist.reshape((X_test_scaled_hist.shape[0],hist_len,22))

input1 = Input(shape=(148,1))
input2 = Input(shape=(hist_len,22))

model=Dense(int(148), activation='relu')(input1)
model = Flatten()(model)

modelhist=(Conv1D(32, int(9), activation='relu')(input2))
modelhist=(MaxPooling1D())(modelhist)                               
modelhist=(Conv1D(32, int(5), activation='relu')(modelhist))
modelhist=(MaxPooling1D())(modelhist)
modelhist=(Dropout(.2))(modelhist)

modelhist=(Flatten())(modelhist)

merged = concatenate([model,modelhist])

output = Dense(128, activation='relu')(merged)
output=Dropout(.2)(output)
output = Dense(32, activation='relu')(output)

output = Dense(1)(output)
merged_model = Model(inputs = [input1,input2], outputs =output )
merged_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
merged_model.summary()

merged_model.fit([X1,X_train_scaled_hist1], y_train_log, validation_data=([X_test1,X_test_scaled_hist1], y_val_log),batch_size=8192, epochs=40)


(1838283, 148) (1838283, 64, 22, 1)
(1838283, 148, 1) (1838283, 64, 22, 1)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 22)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 56, 32)       6368        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 28, 32)       0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 24, 32)       5152        max_pooling1d_1[0][0]            
_________________

In [ ]:
import numpy as np
X_with_log = np.zeros((X.shape[0],148*2))
X_test_with_log = np.zeros((X_test.shape[0],148*2))

X_with_log[:,:148]= X
X_test_with_log[:,:148] = X_test
X_with_log[:,148:]= np.log(abs(X)+.0001)
X_test_with_log[:,148:] = np.log(abs(X_test)+.0001)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers

model.add(Dense(int(148*2), activation='relu'))
model.add(Dense(44, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_with_log, np.array(y_train_log), validation_data=(X_test_with_log, np.array(y_val_log)), batch_size=8192, epochs=6)

In [ ]:
results = model.predict(X_val_scaled)#, y_val_log.values)#, batch_size=128)

#print(X_val_scaled, y_val_log.values, results)
plt.figure(figsize=(200,20))

plt.plot(y_val_log.values)
plt.plot(results)
plt.show()
plt.figure(figsize=(50,15))

plt.plot(y_val_log.values[:1000])
plt.plot(results[:1000])
plt.show()

In [ ]:
plt.figure(figsize=(50,15))

plt.plot(y_val_log.values[:1000])
plt.plot(results[:1000])
plt.show()

In [ ]:



X_log= np.log(abs(X)+.0001)
X_test_log = np.log(abs(X_test)+.0001)


from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers

model.add(Dense(int(148), activation='relu'))
model.add(Dense(22, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(X_log, np.array(y_train_log), validation_data=(X_test_log, np.array(y_val_log)), batch_size=8192, epochs=40)

In [ ]:

from keras.models import Sequential, Model
from keras.layers import Dense, Conv1D, Flatten, Input, LSTM, TimeDistributed, MaxPooling1D
from keras.layers.merge import concatenate

In [ ]:
X1=X.reshape((X.shape[0],148,1))
X_test1=X_test.reshape((X_test.shape[0],148,1))


input1 = Input(shape=(148,1))

model=Dense(int(32), activation='relu')(input1)

merged=TimeDistributed(Flatten())(model)
merged=LSTM(8,activation='relu')(merged)

output = Dense(4, activation='relu')(merged)
output = Dense(1)(output)

merged_model = Model(inputs = input1, outputs =output )
merged_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
merged_model.summary()
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), batch_size=8192, epochs=6)


In [ ]:
X1=X.reshape((X.shape[0],1,148))
X_test1=X_test.reshape((X_test.shape[0],1,148))


input1 = Input(shape=(1,148))

model=Dense(int(148), activation='relu')(input1)

merged=TimeDistributed(Flatten())(input1)
merged=LSTM(32,activation='relu')(merged)

output = Dense(4, activation='relu')(merged)
output = Dense(1)(output)

merged_model = Model(inputs = input1, outputs =output )
merged_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
merged_model.summary()
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), batch_size=8192, epochs=6)


In [ ]:
merged_model.fit(X1, y_train_log, validation_data=(X_test1, y_val_log), batch_size=8192, epochs=34)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()model
#add model layers

model.add(Dense(int(32), activation='relu'))
model.add(Dense(22, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1))

#compile model using accuracy to measure model performance
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

model.fit(X, np.array(y_train_log), validation_data=(X_test, np.array(y_val_log)), batch_size=8192, epochs=40)